In [23]:
import clickhouse_driver
from clickhouse_driver import Client

from pynput import mouse

import pyautogui
import datetime
from datetime import datetime

import pandas as pd
import pandahouse as ph


In [16]:
# connecting to server
connection = clickhouse_driver.connect(
host='localhost',
port=9000,
user='default',
database='mouse_movements')

In [17]:
#creating database
cursor = connection.cursor()
create_database_query = 'CREATE DATABASE IF NOT EXISTS mouse_movements'
cursor.execute(create_database_query)

In [ ]:
#creating table for staging
create_table_query = '''
CREATE TABLE movements (
x UInt16,
y UInt16,
delta_x UInt16,
delta_y UInt16,
client_time_stamp Float32,
button Int8,
target String
)ENGINE=MergeTree() ORDER BY client_time_stamp
'''
cursor.execute(create_table_query)

In [ ]:
#creating table for buffering
create_table_query = '''
CREATE TABLE movements_buffer AS movements 
ENGINE = Buffer(mouse_movements, movements, 
16, 10, 100, 10000, 1000000, 10000000, 100000000)
'''
cursor.execute(create_table_query)

In [ ]:
#mouse catch-click function
client_df = Client('localhost', settings={'use_numpy':True})

prev_x = 0
prev_y = 0
cols = ['x', 'y', 'delta_x', 'delta_y', 'client_time_stamp', 'button', 'target']
clicks = pd.DataFrame(columns=cols)

with mouse.Events() as events:
    for event in events:
        if type(event) == mouse.Events.Click:
            dat = [event.x, event.y, abs(event.x-prev_x), abs(event.y-prev_y), \
                   datetime.now().timestamp(), event.button.value[0],
                   pyautogui.getActiveWindowTitle()]
            clicks.loc[-1] = dat
            #display(clicks)
            client_df.insert_dataframe(f'INSERT INTO mouse_movements.movements_buffer VALUES', clicks)
            prev_x = event.x
            prev_y = event.y

In [ ]:
connection = {'host': 'http://localhost:8123',
              'database': 'mouse_movements',
              'user': 'default',
              'password': ''}

In [ ]:
# count all mouse moves
query_all_movements = 'SELECT COUNT(*) FROM mouse_movements.movements \
                                WHERE delta_x <> 0 OR delta_y <> 0'
df = ph.read_clickhouse(query_all_movements, connection=connection)
display(df)
print('Количество движений:', df.iloc[0,0])

In [ ]:
# count all movements x>1000 and y>1000 group by target
query_xy_over1k = 'SELECT COUNT(*) AS moves_over_1k_px \
                    FROM mouse_movements.movements \
                        WHERE x>1000 \
                        GROUP BY target'
df = ph.read_clickhouse(query_xy_over1k, connection=connection)

In [ ]:
#find biggest moves
query_top_moves = 'SELECT TOP(10) plus(delta_x, delta_y) AS top_moves \
                    FROM mouse_movements.movements \
                    ORDER BY top_moves DESC'
df = ph.read_clickhouse(query_top_moves, connection=connection)

display(df)